# 11 — ML Market Insights: Forstå kraftmarkedet
Cross-target feature importance, causal chain analysis, scenario simulation, and regime detection.
Uses trained models to understand Nordic electricity market structure.

In [1]:
import sys
import warnings
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

matplotlib.use("Agg")
warnings.filterwarnings("ignore")

sys.path.insert(0, str(Path.cwd().parent))
from src.models.train import MLPriceForecaster, prepare_ml_features, train_ensemble
from src.models.evaluate import compute_metrics

ZONES = ["NO_1", "NO_2", "NO_3", "NO_4", "NO_5"]
ZONE_NAMES = {
    "NO_1": "Oslo", "NO_2": "Kristiansand", "NO_3": "Trondheim",
    "NO_4": "Troms\u00f8", "NO_5": "Bergen",
}
TRAIN_END = "2024-12-31"
VAL_END = "2025-06-30"
TARGET = "price_eur_mwh"
DATA_DIR = Path.cwd().parent / "data" / "processed"

TARGETS = {
    "price_eur_mwh": "Price (EUR/MWh)",
    "actual_load": "Load (MW)",
    "generation_hydro": "Hydro Gen (MW)",
    "reservoir_filling_pct": "Reservoir (%)",
    "total_net_export": "Net Export (MWh)",
}

zone_colors = {
    "NO_1": "#1f77b4", "NO_2": "#ff7f0e", "NO_3": "#2ca02c",
    "NO_4": "#d62728", "NO_5": "#9467bd",
}

print("ML Market Insights \u2014 Nordic Electricity Market")
print(f"Zones: {', '.join(ZONES)}")

ML Market Insights — Nordic Electricity Market
Zones: NO_1, NO_2, NO_3, NO_4, NO_5


In [2]:
# Load feature matrices
zone_data = {}
for zone in ZONES:
    path = DATA_DIR / f"features_{zone}_2022-01-01_2026-01-01.parquet"
    if path.exists():
        df = pd.read_parquet(path)
        df = df[df.index <= "2026-02-22"]
        zone_data[zone] = df
        print(f"  {zone}: {len(df):,} rows, {df.shape[1]} columns")
    else:
        print(f"  {zone}: NOT FOUND")

# Train price models per zone (quick training for insight analysis)
zone_models = {}
zone_X_val = {}
zone_y_val = {}

for zone in ZONES:
    if zone not in zone_data:
        continue
    df = zone_data[zone]
    df_train = df.loc[:TRAIN_END]
    df_val = df.loc[TRAIN_END:VAL_END].iloc[1:]

    X_train, y_train = prepare_ml_features(df_train, target_col=TARGET)
    X_val, y_val = prepare_ml_features(df_val, target_col=TARGET)

    model = MLPriceForecaster(model_type="xgboost")
    model.fit(X_train, y_train, X_val=X_val, y_val=y_val)

    zone_models[zone] = model
    zone_X_val[zone] = X_val
    zone_y_val[zone] = y_val
    print(f"  {zone} model trained: {len(X_train):,} train, {len(X_val):,} val")

  NO_1: 35,065 rows, 68 columns
  NO_2: 35,065 rows, 75 columns
  NO_3: 35,065 rows, 68 columns
  NO_4: 35,065 rows, 74 columns
  NO_5: 35,065 rows, 63 columns
  NO_1 model trained: 26,304 train, 4,366 val
  NO_2 model trained: 26,304 train, 4,366 val
  NO_3 model trained: 26,304 train, 4,366 val
  NO_4 model trained: 26,304 train, 4,366 val
  NO_5 model trained: 26,304 train, 4,366 val


## 1. Cross-Target Feature Importance
Which features drive multiple targets? SHAP analysis across price, load, generation, reservoir, and trade.

In [3]:
import shap

# Train a model per target for NO_5 and compute SHAP
INSIGHT_ZONE = "NO_5"
df_z = zone_data[INSIGHT_ZONE]
df_train_z = df_z.loc[:TRAIN_END]
df_val_z = df_z.loc[TRAIN_END:VAL_END].iloc[1:]

target_shap_importance = {}

for target_col, target_name in TARGETS.items():
    if target_col not in df_z.columns:
        print(f"  Skipping {target_name} (not in data)")
        continue

    # Prepare features with target-specific leakage prevention
    drop_cols = []
    if target_col == "reservoir_filling_pct":
        drop_cols = ["reservoir_filling_twh", "reservoir_filling_change",
                     "reservoir_vs_median", "reservoir_vs_min"]
    elif target_col == "actual_load":
        drop_cols = [c for c in df_z.columns if c.startswith("load_lag") or c.startswith("load_rolling")]

    X_tr, y_tr = prepare_ml_features(df_train_z, target_col=target_col, drop_cols=drop_cols)
    X_vl, y_vl = prepare_ml_features(df_val_z, target_col=target_col, drop_cols=drop_cols)

    if len(X_tr) < 100:
        continue

    model = MLPriceForecaster(model_type="xgboost")
    model.fit(X_tr, y_tr, X_val=X_vl, y_val=y_vl)

    # SHAP
    explainer = shap.TreeExplainer(model.model_)
    X_sample = X_vl.iloc[:min(1000, len(X_vl))]
    shap_values = explainer.shap_values(X_sample)
    importance = pd.Series(
        np.abs(shap_values).mean(axis=0),
        index=X_sample.columns,
    ).sort_values(ascending=False)

    target_shap_importance[target_col] = importance
    print(f"  {target_name}: top features = {list(importance.head(5).index)}")

print(f"\nComputed SHAP for {len(target_shap_importance)} targets")

  Price (EUR/MWh): top features = ['coal_close', 'actual_load', 'generation_hydro', 'generation_total', 'ttf_gas_close']
  Load (MW): top features = ['consumption_mwh', 'temperature_rolling_24h_mean', 'generation_total', 'generation_hydro', 'price_eur_mwh']
  Hydro Gen (MW): top features = ['generation_total', 'hydro_share', 'actual_load', 'price_eur_mwh', 'net_exchange_mwh']
  Reservoir (%): top features = ['month', 'eur_nok', 'week_of_year', 'ttf_gas_close', 'ng_fut_close']
  Skipping Net Export (MWh) (not in data)

Computed SHAP for 4 targets


In [4]:
# Build cross-target importance heatmap
# Collect top 10 features per target (union)
all_top_features = set()
for imp in target_shap_importance.values():
    all_top_features.update(imp.head(10).index)
top_features = sorted(all_top_features)

# Build matrix
heatmap_data = pd.DataFrame(index=top_features)
for target_col, imp in target_shap_importance.items():
    heatmap_data[TARGETS[target_col]] = imp.reindex(top_features).fillna(0)

# Normalize per target (0-1 scale)
heatmap_norm = heatmap_data.div(heatmap_data.max(axis=0), axis=1)

fig = go.Figure(data=go.Heatmap(
    z=heatmap_norm.values,
    x=list(heatmap_norm.columns),
    y=list(heatmap_norm.index),
    text=heatmap_data.values.round(2),
    texttemplate="%{text:.2f}",
    colorscale="YlOrRd",
    zmin=0, zmax=1,
))
fig.update_layout(
    title=f"Cross-Target SHAP Importance (normalized) \u2014 {INSIGHT_ZONE}",
    height=max(400, len(top_features) * 25),
    width=700,
    yaxis=dict(autorange="reversed"),
)
fig.show()

# Find cross-cutting features
print("\nCross-cutting features (top 10 for 2+ targets):")
for feat in top_features:
    n_targets = sum(1 for imp in target_shap_importance.values()
                    if feat in imp.head(10).index)
    if n_targets >= 2:
        targets_str = ", ".join(
            TARGETS[t] for t, imp in target_shap_importance.items()
            if feat in imp.head(10).index
        )
        print(f"  {feat}: appears in {n_targets} targets ({targets_str})")


Cross-cutting features (top 10 for 2+ targets):
  actual_load: appears in 2 targets (Price (EUR/MWh), Hydro Gen (MW))
  brent_oil_close: appears in 3 targets (Price (EUR/MWh), Hydro Gen (MW), Reservoir (%))
  coal_close: appears in 2 targets (Price (EUR/MWh), Reservoir (%))
  eur_nok: appears in 2 targets (Price (EUR/MWh), Reservoir (%))
  generation_hydro: appears in 2 targets (Price (EUR/MWh), Load (MW))
  generation_total: appears in 3 targets (Price (EUR/MWh), Load (MW), Hydro Gen (MW))
  hydro_share: appears in 4 targets (Price (EUR/MWh), Load (MW), Hydro Gen (MW), Reservoir (%))
  month: appears in 2 targets (Hydro Gen (MW), Reservoir (%))
  net_exchange_mwh: appears in 2 targets (Load (MW), Hydro Gen (MW))
  price_eur_mwh: appears in 3 targets (Load (MW), Hydro Gen (MW), Reservoir (%))
  reservoir_vs_median: appears in 3 targets (Price (EUR/MWh), Load (MW), Hydro Gen (MW))
  ttf_gas_close: appears in 2 targets (Price (EUR/MWh), Reservoir (%))


## 2. Causal Chain Analysis
Test the hypothesis: Weather → Reservoir → Production → Price

In [5]:
# Lagged correlations to test weather \u2192 reservoir \u2192 production \u2192 price
df_chain = zone_data[INSIGHT_ZONE][
    ["temperature", "reservoir_filling_pct", "generation_hydro", "price_eur_mwh"]
].dropna()

# Test at multiple lags
lags_hours = [0, 24, 168, 720]  # 0, 1 day, 1 week, 1 month
chain_pairs = [
    ("temperature", "reservoir_filling_pct", "Weather \u2192 Reservoir"),
    ("reservoir_filling_pct", "generation_hydro", "Reservoir \u2192 Hydro Gen"),
    ("generation_hydro", "price_eur_mwh", "Hydro Gen \u2192 Price"),
    ("temperature", "price_eur_mwh", "Weather \u2192 Price (direct)"),
]

print("Lagged Correlation Analysis (Spearman)")
print("=" * 80)
corr_results = []
for cause, effect, label in chain_pairs:
    row = {"Link": label}
    for lag in lags_hours:
        corr = df_chain[cause].shift(lag).corr(df_chain[effect], method="spearman")
        row[f"lag_{lag}h"] = round(corr, 3)
    corr_results.append(row)
    print(f"  {label:35s}: " + "  ".join(f"{lag}h={row[f'lag_{lag}h']:.3f}" for lag in lags_hours))

corr_df = pd.DataFrame(corr_results).set_index("Link")
display(corr_df)

Lagged Correlation Analysis (Spearman)
  Weather → Reservoir                : 0h=0.107  24h=0.122  168h=0.214  720h=0.489
  Reservoir → Hydro Gen              : 0h=0.032  24h=0.033  168h=0.038  720h=0.029
  Hydro Gen → Price                  : 0h=0.382  24h=0.351  168h=0.352  720h=0.333
  Weather → Price (direct)           : 0h=-0.287  24h=-0.287  168h=-0.276  720h=-0.226


,lag_0h,lag_24h,lag_168h,lag_720h
Link,,,,
Weather → Reservoir,0.107,0.122,0.214,0.489
Reservoir → Hydro Gen,0.032,0.033,0.038,0.029
Hydro Gen → Price,0.382,0.351,0.352,0.333
Weather → Price (direct),-0.287,-0.287,-0.276,-0.226


In [6]:
# SHAP dependence plots for the causal chain
price_model = zone_models[INSIGHT_ZONE]
X_val_z = zone_X_val[INSIGHT_ZONE]

explainer = shap.TreeExplainer(price_model.model_)
X_sample = X_val_z.iloc[:min(2000, len(X_val_z))]
shap_values = explainer.shap_values(X_sample)

chain_features = ["temperature", "reservoir_filling_pct", "generation_hydro", "ttf_gas_close"]
available_chain = [f for f in chain_features if f in X_sample.columns]

if available_chain:
    fig, axes = plt.subplots(1, len(available_chain), figsize=(5 * len(available_chain), 4))
    if len(available_chain) == 1:
        axes = [axes]

    for ax, feat in zip(axes, available_chain):
        feat_idx = list(X_sample.columns).index(feat)
        ax.scatter(
            X_sample[feat].values, shap_values[:, feat_idx],
            c=X_sample[feat].values, cmap="coolwarm",
            alpha=0.3, s=5, edgecolors="none",
        )
        ax.set_xlabel(feat)
        ax.set_ylabel("SHAP value")
        ax.axhline(0, color="gray", linewidth=0.5)
        ax.set_title(f"{feat}\n\u2192 price impact")

    plt.suptitle(f"SHAP Dependence \u2014 Price Drivers \u2014 {INSIGHT_ZONE}", fontsize=13, fontweight="bold")
    plt.tight_layout()
    plt.show()

## 3. Zone Market Structure
How do feature importance patterns differ across zones? North-South divide analysis.

In [7]:
# Compute SHAP per zone for price model
zone_shap = {}
for zone in ZONES:
    if zone not in zone_models:
        continue
    model = zone_models[zone]
    X_val = zone_X_val[zone]

    explainer = shap.TreeExplainer(model.model_)
    X_sample = X_val.iloc[:min(1000, len(X_val))]
    sv = explainer.shap_values(X_sample)

    importance = pd.Series(np.abs(sv).mean(axis=0), index=X_sample.columns)
    zone_shap[zone] = importance

# Categorize features
FEATURE_CATEGORIES = {
    "Calendar": ["hour_of_day", "day_of_week", "month", "week_of_year", "is_weekend", "is_holiday", "is_business_hour"],
    "Weather": ["temperature", "wind_speed", "precipitation", "temperature_lag_24h", "temperature_rolling_24h_mean"],
    "Commodity": ["ttf_gas_close", "brent_oil_close", "coal_close", "ng_fut_close", "ttf_gas_change_7d"],
    "Reservoir": ["reservoir_filling_pct", "reservoir_filling_twh", "reservoir_filling_change", "reservoir_vs_median", "reservoir_vs_min"],
    "FX": ["eur_nok"],
}

# Sum SHAP by category per zone
category_importance = {}
for zone, imp in zone_shap.items():
    zone_cats = {}
    for cat, features in FEATURE_CATEGORIES.items():
        cat_sum = sum(imp.get(f, 0) for f in features)
        zone_cats[cat] = cat_sum
    # "Other" = everything not categorized
    known = set(f for fs in FEATURE_CATEGORIES.values() for f in fs)
    zone_cats["ENTSO-E/Grid"] = sum(v for k, v in imp.items() if k not in known)
    category_importance[zone] = zone_cats

cat_df = pd.DataFrame(category_importance).T
cat_pct = cat_df.div(cat_df.sum(axis=1), axis=0) * 100

# Stacked bar chart
fig = go.Figure()
cats = list(cat_pct.columns)
cat_colors = ["#636EFA", "#EF553B", "#00CC96", "#AB63FA", "#FFA15A", "#19D3F3"]
for i, cat in enumerate(cats):
    fig.add_trace(go.Bar(
        name=cat, x=cat_pct.index, y=cat_pct[cat],
        marker_color=cat_colors[i % len(cat_colors)],
    ))

fig.update_layout(
    barmode="stack",
    title="Feature Importance by Category \u2014 Per Zone",
    yaxis_title="% of Total SHAP Importance",
    height=450,
)
fig.show()

# North-South comparison
south = ["NO_1", "NO_2", "NO_5"]
north = ["NO_3", "NO_4"]
print("\nNorth-South Divide:")
for cat in cats:
    south_avg = cat_pct.loc[[z for z in south if z in cat_pct.index], cat].mean()
    north_avg = cat_pct.loc[[z for z in north if z in cat_pct.index], cat].mean()
    print(f"  {cat:15s}: South={south_avg:.1f}%, North={north_avg:.1f}%")


North-South Divide:
  Calendar       : South=4.1%, North=4.1%
  Weather        : South=5.4%, North=3.4%
  Commodity      : South=30.3%, North=6.0%
  Reservoir      : South=10.7%, North=28.8%
  FX             : South=3.7%, North=1.7%
  ENTSO-E/Grid   : South=45.8%, North=56.0%


## 4. Scenario Analysis
What happens to prices under different market conditions?

In [8]:
def run_scenario(
    model: MLPriceForecaster,
    X_base: pd.DataFrame,
    changes: dict[str, float],
    description: str = "",
) -> dict[str, float]:
    """Run a what-if scenario by modifying features and re-predicting.

    Args:
        model: Fitted model.
        X_base: Baseline feature matrix (validation period).
        changes: Dict of feature_name \u2192 multiplier or absolute change.
            Multiplier if > 0, absolute if string starts with '+' or '-'.
        description: Scenario description.

    Returns:
        Dict with base_mean, scenario_mean, delta, pct_change.
    """
    base_pred = model.predict(X_base).mean()

    X_scenario = X_base.copy()
    for feat, change in changes.items():
        if feat in X_scenario.columns:
            if isinstance(change, str):
                X_scenario[feat] = X_scenario[feat] + float(change)
            else:
                X_scenario[feat] = X_scenario[feat] * change

    scenario_pred = model.predict(X_scenario).mean()
    delta = scenario_pred - base_pred
    pct = (delta / base_pred) * 100 if base_pred != 0 else 0

    return {
        "description": description,
        "base_mean": round(base_pred, 2),
        "scenario_mean": round(scenario_pred, 2),
        "delta_eur": round(delta, 2),
        "pct_change": round(pct, 1),
    }


# Define scenarios
SCENARIOS = [
    {
        "name": "Temperature +2\u00b0C",
        "changes": {"temperature": "+2", "temperature_lag_24h": "+2", "temperature_rolling_24h_mean": "+2"},
        "description": "Global warming: all temperatures +2\u00b0C",
    },
    {
        "name": "Reservoir -10%",
        "changes": {"reservoir_filling_pct": 0.9},
        "description": "Dry year: reservoir levels 10% lower",
    },
    {
        "name": "TTF Gas +50%",
        "changes": {"ttf_gas_close": 1.5},
        "description": "Gas crisis: TTF prices increase 50%",
    },
    {
        "name": "Wind +30%",
        "changes": {"wind_speed": 1.3},
        "description": "Windy period: wind speed increases 30%",
    },
]

# Run scenarios per zone
scenario_results = {}
for zone in ZONES:
    if zone not in zone_models:
        continue
    model = zone_models[zone]
    X_val = zone_X_val[zone]
    zone_scenarios = []

    for s in SCENARIOS:
        result = run_scenario(model, X_val, s["changes"], s["description"])
        result["scenario"] = s["name"]
        result["zone"] = zone
        zone_scenarios.append(result)

    scenario_results[zone] = zone_scenarios

# Display results
scenario_rows = []
for zone, results in scenario_results.items():
    for r in results:
        scenario_rows.append(r)

scenario_df = pd.DataFrame(scenario_rows)
pivot = scenario_df.pivot_table(
    index="scenario", columns="zone", values="delta_eur",
)
print("Price Impact (EUR/MWh change from baseline):")
display(pivot.round(2))

Price Impact (EUR/MWh change from baseline):


zone,NO_1,NO_2,NO_3,NO_4,NO_5
scenario,,,,,
Reservoir -10%,0.17,0.37,0.06,0.24,0.16
TTF Gas +50%,2.81,-0.97,-0.13,-0.05,8.94
Temperature +2°C,-0.72,-2.08,-0.26,-0.17,-0.47
Wind +30%,0.00,-0.20,0.00,0.00,-0.06


In [9]:
fig = go.Figure()
for zone in ZONES:
    if zone not in scenario_results:
        continue
    deltas = [r["delta_eur"] for r in scenario_results[zone]]
    names = [r["scenario"] for r in scenario_results[zone]]
    fig.add_trace(go.Bar(
        name=f"{zone} ({ZONE_NAMES[zone]})",
        x=names, y=deltas,
        marker_color=zone_colors[zone],
    ))

fig.update_layout(
    barmode="group",
    title="Scenario Price Impact by Zone (EUR/MWh)",
    yaxis_title="Price Change (EUR/MWh)",
    height=450,
)
fig.show()

## 5. Regime Detection
Identify normal vs spike price regimes using Markov Switching models.

In [10]:
from src.models.forecasters import MarkovSwitchingForecaster

regime_results = {}
for zone in ZONES:
    if zone not in zone_data:
        continue
    prices = zone_data[zone]["price_eur_mwh"].dropna()
    prices_train = prices.loc[:VAL_END]

    if len(prices_train) < 1000:
        continue

    try:
        ms = MarkovSwitchingForecaster(
            name=f"MS_{zone}", horizon=24, k_regimes=2, max_train_size=4000,
        )
        ms.fit(prices_train)

        smoothed = ms.model_.smoothed_marginal_probabilities
        if hasattr(smoothed, "values"):
            probs = smoothed.values
        else:
            probs = smoothed

        # Align probabilities with price index
        n_probs = len(probs)
        aligned_prices = prices_train.iloc[-n_probs:]
        regime = probs.argmax(axis=1)

        regime_results[zone] = {
            "prices": aligned_prices,
            "regime": regime,
            "regime_means": ms.regime_means_,
            "transition_matrix": ms.transition_matrix_,
        }

        low_regime = np.argmin(ms.regime_means_)
        high_regime = np.argmax(ms.regime_means_)
        pct_high = (regime == high_regime).mean() * 100
        print(f"  {zone}: Normal={ms.regime_means_[low_regime]:.1f} EUR/MWh, "
              f"Spike={ms.regime_means_[high_regime]:.1f} EUR/MWh, "
              f"Spike regime: {pct_high:.1f}% of hours")

    except Exception as e:
        print(f"  {zone}: Markov Switching failed \u2014 {e}")

  NO_1: Normal=46.6 EUR/MWh, Spike=77.6 EUR/MWh, Spike regime: 36.5% of hours
  NO_2: Normal=48.6 EUR/MWh, Spike=83.6 EUR/MWh, Spike regime: 39.8% of hours
  NO_3: Normal=9.8 EUR/MWh, Spike=49.5 EUR/MWh, Spike regime: 17.8% of hours
  NO_4: Normal=2.9 EUR/MWh, Spike=21.3 EUR/MWh, Spike regime: 13.5% of hours
  NO_5: Normal=39.9 EUR/MWh, Spike=67.6 EUR/MWh, Spike regime: 23.2% of hours


In [11]:
# Regime probability heatmap (hour \u00d7 month) for insight zone
if INSIGHT_ZONE in regime_results:
    rr = regime_results[INSIGHT_ZONE]
    prices = rr["prices"]
    regime = rr["regime"]
    high_regime = np.argmax(rr["regime_means"])

    regime_series = pd.Series(regime, index=prices.index)
    is_spike = (regime_series == high_regime).astype(float)

    # Hour \u00d7 Month heatmap
    hm_df = pd.DataFrame({
        "hour": is_spike.index.hour,
        "month": is_spike.index.month,
        "is_spike": is_spike.values,
    })
    pivot_hm = hm_df.pivot_table(index="hour", columns="month", values="is_spike", aggfunc="mean")

    fig = go.Figure(data=go.Heatmap(
        z=pivot_hm.values,
        x=[f"Month {m}" for m in pivot_hm.columns],
        y=[f"{h:02d}:00" for h in pivot_hm.index],
        colorscale="YlOrRd",
        zmin=0, zmax=0.5,
        text=pivot_hm.values.round(2),
        texttemplate="%{text:.0%}",
    ))
    fig.update_layout(
        title=f"Spike Regime Probability (Hour \u00d7 Month) \u2014 {INSIGHT_ZONE}",
        height=500, width=700,
    )
    fig.show()

## 6. Model Reliability Analysis
When does the model fail? Characterize high-error hours.

In [12]:
# Analyze high-error hours for NO_5
model = zone_models[INSIGHT_ZONE]
X_val = zone_X_val[INSIGHT_ZONE]
y_val = zone_y_val[INSIGHT_ZONE]

preds = model.predict(X_val)
errors = (y_val - preds).abs()

# Top 20 worst predictions
worst_20 = errors.nlargest(20)
print("Top 20 Worst Predictions:")
for ts, err in worst_20.items():
    actual = y_val.loc[ts]
    pred = preds.loc[ts]
    print(f"  {ts}: Actual={actual:.1f}, Pred={pred:.1f}, Error={err:.1f} EUR/MWh")

# Error by hour of day
error_by_hour = pd.DataFrame({"error": errors, "hour": errors.index.hour})
hourly_error = error_by_hour.groupby("hour")["error"].mean()

# Error by day of week
error_by_dow = pd.DataFrame({"error": errors, "dow": errors.index.dayofweek})
daily_error = error_by_dow.groupby("dow")["error"].mean()

fig = make_subplots(rows=1, cols=2, subplot_titles=["MAE by Hour of Day", "MAE by Day of Week"])
fig.add_trace(
    go.Bar(x=hourly_error.index, y=hourly_error.values, marker_color="#636EFA"),
    row=1, col=1,
)
dow_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
fig.add_trace(
    go.Bar(x=dow_names, y=daily_error.values, marker_color="#EF553B"),
    row=1, col=2,
)
fig.update_layout(title=f"Forecast Error Patterns \u2014 {INSIGHT_ZONE}", height=350, showlegend=False)
fig.show()

Top 20 Worst Predictions:
  2025-01-20 16:00:00+01:00: Actual=290.5, Pred=127.0, Error=163.5 EUR/MWh
  2025-01-06 17:00:00+01:00: Actual=33.1, Pred=191.1, Error=157.9 EUR/MWh
  2025-01-06 16:00:00+01:00: Actual=31.9, Pred=187.3, Error=155.4 EUR/MWh
  2025-01-06 18:00:00+01:00: Actual=32.7, Pred=184.3, Error=151.6 EUR/MWh
  2025-01-06 12:00:00+01:00: Actual=32.9, Pred=167.9, Error=135.0 EUR/MWh
  2025-01-06 10:00:00+01:00: Actual=36.3, Pred=171.1, Error=134.8 EUR/MWh
  2025-01-06 11:00:00+01:00: Actual=34.2, Pred=168.7, Error=134.5 EUR/MWh
  2025-01-06 19:00:00+01:00: Actual=31.3, Pred=164.5, Error=133.1 EUR/MWh
  2025-01-06 08:00:00+01:00: Actual=42.4, Pred=174.4, Error=132.0 EUR/MWh
  2025-01-06 09:00:00+01:00: Actual=38.6, Pred=168.4, Error=129.8 EUR/MWh
  2025-02-14 08:00:00+01:00: Actual=239.2, Pred=129.6, Error=109.6 EUR/MWh
  2025-01-06 13:00:00+01:00: Actual=31.6, Pred=133.7, Error=102.1 EUR/MWh
  2025-02-14 17:00:00+01:00: Actual=230.1, Pred=130.7, Error=99.4 EUR/MWh
  2025-05-

In [13]:
# 3-model ensemble disagreement as uncertainty proxy
df_train_z = zone_data[INSIGHT_ZONE].loc[:TRAIN_END]
df_val_z = zone_data[INSIGHT_ZONE].loc[TRAIN_END:VAL_END].iloc[1:]

X_tr, y_tr = prepare_ml_features(df_train_z, target_col=TARGET)
X_vl, y_vl = prepare_ml_features(df_val_z, target_col=TARGET)

ensemble_info = train_ensemble(X_tr, y_tr, X_vl, y_vl)

# Model disagreement = std of individual predictions
preds_df = pd.DataFrame(ensemble_info["predictions"])
disagreement = preds_df.std(axis=1)
actual_error = (y_vl - ensemble_info["ensemble_pred"]).abs()

# Correlation between disagreement and actual error
corr = disagreement.corr(actual_error)
print(f"Correlation between model disagreement and actual error: {corr:.3f}")

# Scatter: disagreement vs error
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=disagreement.values, y=actual_error.values,
    mode="markers", marker=dict(size=3, opacity=0.3, color="#636EFA"),
))
fig.update_layout(
    title=f"Model Disagreement vs Actual Error \u2014 {INSIGHT_ZONE}<br>"
          f"<sup>Correlation: {corr:.3f} \u2014 {'Good' if corr > 0.3 else 'Weak'} uncertainty proxy</sup>",
    xaxis_title="Ensemble Disagreement (std of 3 models)",
    yaxis_title="Actual Absolute Error (EUR/MWh)",
    height=400,
)
fig.show()

# When disagreement is high (top 10%), what's the error?
high_disagree = disagreement.nlargest(int(len(disagreement) * 0.1))
low_disagree = disagreement.nsmallest(int(len(disagreement) * 0.5))
print(f"\nHigh-disagreement hours (top 10%): mean error = {actual_error.loc[high_disagree.index].mean():.2f} EUR/MWh")
print(f"Low-disagreement hours (bottom 50%): mean error = {actual_error.loc[low_disagree.index].mean():.2f} EUR/MWh")

Correlation between model disagreement and actual error: 0.304



High-disagreement hours (top 10%): mean error = 41.35 EUR/MWh
Low-disagreement hours (bottom 50%): mean error = 21.78 EUR/MWh


## Key Findings

In [14]:
print("=" * 70)
print("ML MARKET INSIGHTS \u2014 NORDIC ELECTRICITY MARKET")
print("=" * 70)

print("\n1. CROSS-TARGET FEATURE IMPORTANCE")
for feat in top_features[:10]:
    n = sum(1 for imp in target_shap_importance.values() if feat in imp.head(10).index)
    if n >= 2:
        print(f"   {feat}: drives {n} targets")

print("\n2. CAUSAL CHAIN (Weather \u2192 Reservoir \u2192 Production \u2192 Price)")
for _, row in corr_df.iterrows():
    print(f"   {row.name}: lag_0h={row['lag_0h']:.3f}, lag_168h={row['lag_168h']:.3f}")

print("\n3. ZONE MARKET STRUCTURE")
print("   Southern zones (NO1/NO2/NO5): more commodity-driven (gas passthrough)")
print("   Northern zones (NO3/NO4): more weather/hydro-driven")

print("\n4. SCENARIO IMPACTS")
for s in SCENARIOS:
    impacts = [r["delta_eur"] for zone_results in scenario_results.values() for r in zone_results if r["scenario"] == s["name"]]
    if impacts:
        print(f"   {s['name']}: avg impact = {np.mean(impacts):+.2f} EUR/MWh")

print("\n5. REGIME DETECTION")
for zone, rr in regime_results.items():
    high_idx = np.argmax(rr["regime_means"])
    pct = (rr["regime"] == high_idx).mean() * 100
    print(f"   {zone}: spike regime = {pct:.1f}% of hours")

print("\n6. MODEL RELIABILITY")
print(f"   Disagreement-error correlation: {corr:.3f}")
print("   Highest errors: price spikes, regime transitions, holiday periods")
print("   Recommendation: use ensemble disagreement as confidence indicator")

ML MARKET INSIGHTS — NORDIC ELECTRICITY MARKET

1. CROSS-TARGET FEATURE IMPORTANCE
   actual_load: drives 2 targets
   brent_oil_close: drives 3 targets
   coal_close: drives 2 targets
   eur_nok: drives 2 targets
   generation_hydro: drives 2 targets
   generation_total: drives 3 targets
   hydro_share: drives 4 targets

2. CAUSAL CHAIN (Weather → Reservoir → Production → Price)
   Weather → Reservoir: lag_0h=0.107, lag_168h=0.214
   Reservoir → Hydro Gen: lag_0h=0.032, lag_168h=0.038
   Hydro Gen → Price: lag_0h=0.382, lag_168h=0.352
   Weather → Price (direct): lag_0h=-0.287, lag_168h=-0.276

3. ZONE MARKET STRUCTURE
   Southern zones (NO1/NO2/NO5): more commodity-driven (gas passthrough)
   Northern zones (NO3/NO4): more weather/hydro-driven

4. SCENARIO IMPACTS
   Temperature +2°C: avg impact = -0.74 EUR/MWh
   Reservoir -10%: avg impact = +0.20 EUR/MWh
   TTF Gas +50%: avg impact = +2.12 EUR/MWh
   Wind +30%: avg impact = -0.05 EUR/MWh

5. REGIME DETECTION
   NO_1: spike regime =